In [153]:
import numpy as np 

In [154]:
def convert_data(path:str, line_start:int, line_end:int=-4, num_type:str="int") -> np.array:
    """
    path:文件路径,
    line_start:数字开始行,
    line_end:数字结束行,
    num_type:数字类型,
    """
    num_type_dict = {
        "int":int,
        "float":float,
    }
    with open(path, 'r') as file:
        lines = file.readlines()[line_start:line_end]
        data = []
        for line in lines:
            # 替换'4(28 617 618 29)\n'中的括号与空格为逗号
            line = line.strip().replace("(", " ").replace(")", " ").split()
            # 替换['4', '28', '617', '618', '29']当中字符串为数字
            line = [num_type_dict[num_type](l) for l in line]
            data.append(line)
        data = np.array(data)
    return data 

In [155]:
faces = convert_data("./faces", 19)
faces = faces[:, 1:]

In [156]:
neighbours = convert_data("./neighbour", 20)

In [157]:
owners = convert_data("./owner", 20)

In [158]:
points = convert_data("./points", 19, num_type="float")
points = points * 1000

In [159]:
boundary = {  
    "inlet": {  
        "type": "patch",  
        "nFaces": 30,  
        "startFace": 24170  
    },  
    "outlet": {  
        "type": "patch",  
        "nFaces": 57,  
        "startFace": 24200  
    },  
    "upperWall": {  
        "type": "wall",  
        "inGroups": "List<word> 1(wall)",  
        "nFaces": 223,  
        "startFace": 24257  
    },  
    "lowerWall": {  
        "type": "wall",  
        "inGroups": "List<word> 1(wall)",  
        "nFaces": 250,  
        "startFace": 24480  
    },  
    "frontAndBack": {  
        "type": "empty",  
        "inGroups": "List<word> 1(empty)",  
        "nFaces": 24450,  
        "startFace": 24730  
    }  
}

In [160]:
class Node():
    def __init__(
            self, 
            centriod:np.array=np.zeros(3), # 节点形心坐标，即节点的坐标x,y,z
            index:int=0,  # 节点整体标识（编号，编码）
            iFaces:np.array=np.zeros(3), # 节点被哪些面所享有，这些面的标识列表
            iElement:np.array=np.zeros(3), # 节点被哪些单元所享有，这些单元的标识列表
            ) -> None:
        self.centriod = centriod
        self.index = index
        self.iFaces = iFaces
        self.iElement = iElement 
class Face():
    def __init__(
            self, 
            iNodes:np.array=np.zeros(3), # 构成该面的节点列表
            index:int=0, # 该面的标识
            iOwner:int=0, # 该面owner单元标识
            iNeighbour:int=0, # 该面neighbour单元标识（若为边界面，则该标识为-1）
            centriod:np.array=np.zeros(3), # 该面形心坐标x,y,z
            Sf:np.array=np.zeros(3), # 该面的面积矢量Sx,Sy,Sz
            area:float=0.0, # 该面的面积S
            Cf:np.array=np.zeros(3), # 该面所属单元形心到该面形心的距离矢量Cf
            geoDiff:float=0.0, # 面几何扩散系数 gDiff_f = Ef / CF
            T:np.array=np.zeros(3), # 面所属单元和邻近单元形心之间的距离矢量CF？（感觉更像是Tf=Sf-CF为非正交修正矢量，见第8章内容）
            gf:float=0.0, # 面插值中的几何权重系数gf
            walldist:int=0, # 面所属单元形心到壁面的垂直距离（某些湍流模型中会用到）
            iOwnerNeighbourCoef:np.array=np.array([0]), # ?
            iNeighbourOwnerCoef:np.array=np.array([0]), # ?
            ):        
        self.iNodes = iNodes
        self.index = index
        self.iOwner = iOwner 
        self.iNeighbour = iNeighbour
        self.centriod = centriod
        self.Sf = Sf 
        self.area = area 
        self.Cf = Cf 
        self.geoDiff = geoDiff 
        self.T = T 
        self.gf = gf 
        self.walldist = walldist
        self.iOwnerNeighbourCoef = iOwnerNeighbourCoef
        self.iNeighbourOwnerCoef = iNeighbourOwnerCoef
    # def initialize_data(self):
class Element():
    def __init__(
            self,
            index:int=0, # 该单元标识
            iNeighbour:np.array=np.zeros(3), # 该单元的邻近单元（与该单元共享面的那些单元）标识列表
            iFaces:np.array=np.zeros(3), # 构成该单元的面标识列表
            iNodes:np.array=np.zeros(3), # 构成该单元的节点标识列表
            volume:float=0.0, # 该单元体积
            faceSign:np.array=np.zeros(3), # 该单元的构成面是否为其owner面（==1）或neighbour（==-1）
            numberOfNeighbour:int=0, # 该单元邻近单元数目
            centroid:np.array=np.zeros(3), # 该单元的形心坐标x,y,z
            ):
        self.index = index 
        self.iNeighbour = iNeighbour
        self.iFaces = iFaces 
        self.iNodes = iNodes 
        self.volume = volume 
        self.faceSign = faceSign 
        self.numberOfNeighbour = numberOfNeighbour
        self.centroid = centroid
class Boundary():
    def __init__(
            self,
            userName:str=None,
            index:int=0,
            type_:str=None,
            numberOfBFaces:int=0,
            startFace:int=0,
            ) -> None:
        self.userName = userName 
        self.index = index 
        self.type = type_ 
        self.numberOfBFaces = numberOfBFaces
        self.startFace = startFace

In [161]:
# 对面的各种量进行计算
for face in faces:
    # face 面的角点标识
    theNumberOfFaceNodes = len(face) # 某个面对应角点的个数
    # 计算面的大概几何中心
    local_center = np.zeros(3)
    for iNode in face:
        local_center += points[iNode]
    local_center /= theNumberOfFaceNodes

    centroid = np.zeros(3) # 初始化该面形心
    Sf = np.zeros(3) # 初始化该面面矢量
    area = 0 # 初始化该面面积

    for iTriangle in range(theNumberOfFaceNodes):
        # 计算该面被分割成的小三角形的形心、面矢量、面积
        point1 = centroid
        point2 = points[face[iTriangle], :]
        if iTriangle < theNumberOfFaceNodes-1: # python从0开始索引
            point3 = points[face[iTriangle+1], :]
        else:
            point3 = points[face[0], :]
        local_centroid = (point1+point2+point3) / 3 # 计算小三角形形心
        local_Sf = (1/2) * np.cross(point2-point1, point3-point1) # 计算小三角形面积矢量
        local_area = np.linalg.norm(local_Sf)

        centroid = centroid + local_area*local_centroid
        Sf = Sf + local_Sf 
        area = area + local_area 
    centroid = centroid / area

In [162]:
with open("./owner", 'r') as f:
    file = f.readlines()
note = file[11].strip().replace('"', '').replace(';', '').replace(':', '').split()
the_number_of_elements = int(note[4])
the_number_of_interior_faces = int(note[8])
the_number_of_faces = int(note[6])

In [163]:
# file[11].strip().replace('"', '').replace(';', '').replace(':', '').split()

In [164]:
"""
面的邻居单元（neighbour）标识列表
面的所属（owner）单元标识
"""
# Element-neighbours indices, Element-faces indices, Element-node indices
element_neighbours = [[] for _ in range(the_number_of_elements)] # 单元旁边的单元index
element_faces = [[] for _ in range(the_number_of_elements)] # 单元旁边的面index

for iFace in range(the_number_of_interior_faces):
    own = owners[iFace][0]
    nei = neighbours[iFace][0]
    # 面相邻的单元1添加到相邻的单元2那里
    element_neighbours[own].append(nei)
    element_neighbours[nei].append(own)
    # 单元旁边的面
    element_faces[own].append(iFace)
    element_faces[nei].append(iFace)

for iFace in range(the_number_of_interior_faces, the_number_of_faces):
    own = owners[iFace][0]
    element_faces[own].append(iFace)

In [191]:
def unique(ls_1D):
    ls_temp = []
    for l in ls_1D:
        if l in ls_temp:
            continue
        ls_temp.append(l)
    return ls_temp 

In [192]:
element_nodes = [[] for _ in range(the_number_of_elements)] # 单元旁边的点index

for iElement in range(the_number_of_elements):
    for face_index in element_faces[iElement]:
        element_nodes_ = list(faces[face_index]) # faces 对应 uFVM中的faceNodes
        element_nodes[iElement].extend(element_nodes_)
    element_nodes[iElement] = unique(element_nodes[iElement])

In [203]:
# Element Anb coefficients indices
upper_and_coeff_index = np.zeros((the_number_of_interior_faces, 1))
lower_and_coeff_index = np.zeros((the_number_of_interior_faces, 1))

for iElement in range(the_number_of_elements):
    iNb = 1
    for face_index in element_faces[iElement]:
        # 跳过边界面
        if face_index > the_number_of_interior_faces-1:
            continue
        # own, nei 就是某面两个的相邻单元
        own = owners[face_index]
        nei = neighbours[face_index]
        if iElement == own:
            upper_and_coeff_index[face_index] = iNb 
        elif iElement == nei:
            lower_and_coeff_index[face_index] = iNb 
        iNb += 1

In [218]:
def cfd_invert_connectivity(the_connectivity_array):
    the_inverted_size = 0
    the_inverted_size_ls = []
    for arr in the_connectivity_array:
        the_inverted_size = max(arr)
        the_inverted_size_ls.extend(the_inverted_size)
    for arr in the_inverted_size_ls:
        the_inverted_size = max(arr)
    the_inverted_connectivity_array = [[] for _ in range(the_inverted_size)]
    for i in range(len(the_connectivity_array)):
        for j in range(len(the_connectivity_array[i])):
            the_inverted_connectivity_array[i].append(the_connectivity_array[j])
    return the_inverted_connectivity_array

In [219]:
# Invert connectivity
node_elements = cfd_invert_connectivity(element_nodes)
node_faces    = cfd_invert_connectivity(faces)

TypeError: 'numpy.int32' object is not iterable

In [220]:
[0 for _ in range(3)]

[0, 0, 0]